# Mixtral 7B Round Table

In [ ]:
%cd /content/drive/MyDrive/NLP/semevaltask9

/content/drive/MyDrive/NLP/semevaltask9


In [ ]:
import torch
import numpy as np
import pandas as pd
import random
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

In [ ]:
GLOBAL_SEED = 255

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


env: CUBLAS_WORKSPACE_CONFIG=:4096:8


## Use hugging face chat

In [ ]:
!pip install hugchat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
from hugchat import hugchat
from hugchat.login import Login

# Log in to huggingface and grant authorization to huggingchat
sign = Login(mail, password)
cookies = sign.login()

# Save cookies to the local directory
cookie_path_dir = "./cookies_snapshot"
sign.saveCookiesToDir(cookie_path_dir)

'./cookies_snapshot/baktash.ansari1381@gmail.com.json'

In [ ]:
chatbot = hugchat.ChatBot(cookies=cookies.get_dict())

In [ ]:
# Create a new conversation with mixtral
chatbot.switch_llm(0)
id = chatbot.new_conversation()
chatbot.change_conversation(id)

### Load datasets

In [ ]:
# Import Dataset
data = np.load("./Data/SP_train.npy", allow_pickle = True ).tolist()
SP_train = pd.DataFrame(data)
data = np.load("./Data/SP_test.npy", allow_pickle = True ).tolist()
SP_test = pd.DataFrame(data)
data = np.load("./Data/SP_test_answer.npy", allow_pickle = True ).tolist()
SP_test_answer = pd.DataFrame(data)

data = np.load("./Data/WP_train.npy", allow_pickle = True ).tolist()
WP_train = pd.DataFrame(data)
data = np.load("./Data/WP_test.npy", allow_pickle = True ).tolist()
WP_test = pd.DataFrame(data)
data = np.load("./Data/WP_test_answer.npy", allow_pickle = True ).tolist()
WP_test_answer = pd.DataFrame(data)

data = np.load("./Data/sentence_puzzle.npy", allow_pickle = True ).tolist()
SP = pd.DataFrame(data)
data = np.load("./Data/word_puzzle.npy", allow_pickle = True ).tolist()
WP = pd.DataFrame(data)

In [ ]:
for index, row in SP_test.iterrows():
  SP_test.at[index, 'label'] = int(SP_test_answer.loc[index][1])

In [ ]:
for index, row in WP_test.iterrows():
  WP_test.at[index, 'label'] = int(WP_test_answer.loc[index][1])

## Initial Round

### Create Prompt

In [ ]:
prompt = """Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution:
Reasoning:
Confidence:

Question: """

In [ ]:
def generate_prompt(item, prompt):

  labels = ["A", "B", "C", "D"]
  prompt += f'{item["question"]}\n'
  for index, choice in enumerate(item["choice_list"]):
    prompt += f'{labels[index]} : {choice}\n'
  return prompt

In [ ]:
SP_test.loc[90]

question       During a 5-day competition, a professional fis...
choice_list                 [Two., One., Three., None of above.]
label                                                        3.0
Name: 90, dtype: object

In [ ]:
for index, row in SP_test.iterrows():
  print(index + 1,": ",end = "")
  print(generate_prompt(row, prompt))
  print("#################################################")

1 : Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution:
Reasoning:
Confidence:

Question: In a small village, two farmers are working in their fields - a diligent farmer and a lazy farmer. The hardworking farmer is the son of the lazy farmer, but the lazy farmer is not the father of the hardworking farmer. Can you explain this unusual relationship?
A : The lazy farmer is his mother.
B : The lazy farmer is not a responsible father as he is lazy.
C : The diligent farmer devoted himself to the farm and gradually forgot his father.
D : None of above.

#################################################
2 : Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution:
Re

### Run inference

In [ ]:
mixtralResultsR0

In [ ]:
import time
# mixtralResultsR0 = []
for index, row in SP_test.iterrows():
  if index + 1 < 66:
    continue
  query_result = chatbot.query(generate_prompt(row, prompt))
  print(index + 1)
  print(query_result)
  mixtralResultsR0.append({"id" : index+1, "response" : query_result.text})
  print()
  time.sleep(2)

66
 Solution: A, You need to drop the egg gently.

Reasoning: Raw eggs are fragile and susceptible to breaking when subjected to force. By dropping the egg gently, there is a greater chance that the eggshell will not fracture upon contact with the concrete floor due to reduced impact forces.

Confidence: 0.9 (Highly Likely)

67
 Solution: C, The man is blind and is reading braille.

Reasoning: Since there is no source of light in the house, it is impossible for a sighted person to read a regular printed book. However, if the man is blind, he could be reading Braille dots, which do not require light to read.

Confidence: 0.95 (very confident)

68
 Solution: B, It was driven over in winter, when the lake was frozen.

Reasoning: The question states that there has never been a bridge connecting the island to the mainland and the tractor was not transported to the island by boat or by air, nor was it built on the island. This implies that the tractor must have crossed the lake when it was f

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/hugchat/hugchat.py", line 704, in _stream_query
    yield obj
GeneratorExit
Exception ignored in: <generator object ChatBot._stream_query at 0x7e1f5eab5b60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/hugchat/hugchat.py", line 730, in _stream_query
    raise exceptions.ChatError(f"Failed to parse response: {res}")
hugchat.exceptions.ChatError: Failed to parse response: {"message":"You are sending too many messages. Try again later."}


  Solution: A, The coffee shops and his home are on a slope, which allows him to walk downhill easily both to work and back home.

Reasoning: The worker goes to the first coffee shop in the morning and the second one in the evening. Walking downhill conserves energy compared to climbing uphill, especially when doing so twice a day. The question does not specify the distances to work or the effort required to travel there, only the locations of the coffee shops.

Confidence: 0.75. The answer is plausible, but the question does not fully define the layout of the streets or the topography.

85
 Solution: D, None of above.

Reasoning: Though the answer options attempt to provide possible explanations, they don't fully address the situation presented in the question. The question asks why the restaurant informed the customer that all tables were booked, even though there were no customers visible later.

One possible explanation is that the restaurant oversold their reservations or double-b

In [ ]:
# save dataframe:
mixtralR0 = pd.DataFrame(mixtralResultsR0)
mixtralR0.to_csv("./MixtralR0.csv")

,id,response
0,1,"Solution: A, The lazy farmer is his mother.\n..."
1,2,"Solution: C, Romeo and Juliet are fish. The r..."
2,3,"Solution: C, It happens every year.\n\nReason..."
3,4,"Solution: B, The captain.\n\nReasoning: Gener..."
4,5,"Solution: C, Five people.\n\nReasoning: We kn..."
...,...,...
115,116,Solution: To calculate the number of fish cau...
116,117,"Solution: C, The internet was disconnected so..."
117,118,"Solution:\nHe has wedded numerous women, but ..."
118,119,Solution:\nNumber of legs on the floor = Numb...


In [ ]:
mixtralResultsR0[0]['response'].split('Confidence: ')[1]

'With the available data, I am moderately confident (0.7) that my proposed solution accurately represents the intended meaning behind the riddle.'

In [ ]:
# create structural data frame:
Structural_Mixtral_R0 = []

for index, item in enumerate(mixtralResultsR0):
  print(index)
  print(item['response'].split('Solution: '))
  if index == 117:
    solution = 'B'
    confidence = '0.9'
    reasoning = 'Given the statement, the person in consideration performs wedding ceremonies for several women ("numerous") without getting married himself. Marriage officiants preside over wedding ceremonies for others without participating in matrimonial bondage themselves.\n\nConsidering the available options, a preacher often performs religious ceremonies like weddings and is less likely to marry multiple times himself.'
  elif index == 118:
    solution = 'D'
    confidence = '0.95'
    reasoning = 'We can analyze this question by calculating the number of legs contributed by each entity separately. First, consider the bed, which has four legs. Humans have two legs. Animals vary in leg count, but it is unnecessary to calculate individual contributions since the question seeks only the quantity of legs touching the floor.\n\nCounting the number of entities resting on the bed is irrelevant because they do not add any extra legs to the floor. Instead, focus on the bed itself and the person lying on it. The doves flying overhead pose no bearing on the calculation, as they do not interact with the floor.'
  else:
    solution = item['response'].split('Solution: ')[1][0]
    reasoning = item['response'].split('Reasoning: ')[1].split('Confidence')[0]

    if index == 64:
      confidence = item['response'].split('Confidence Level: ')[1]
    else:
      confidence = item['response'].split('Confidence: ')[1]
  Structural_Mixtral_R0.append({"Solution" : solution, "Reasoning" : reasoning, "Confidence" : confidence})

Structural_Mixtral_R0

0
[' ', "A, The lazy farmer is his mother.\n\nReasoning: Let us analyze the given information:\n1. There are two farmers: one diligent and the other lazy.\n2. The lazy farmer has a child who turned out to be a hardworking farmer.\n3. It is stated that the lazy farmer isn't the father of the hardworking farmer.\n4. Since they are farmers living together in a household, it suggests a close familial bond.\n\nConsidering the language used and avoiding assumptions outside what was directly revealed, the laziest interpretation aligns with answer choice A – the lazy farmer is indeed the mother of the diligent farmer. As long as the genders aren't specified otherwise, assuming either gender role doesn't violate the text. Hence, without further context, selecting A satisfies all conditions while excluding unnecessary speculation.\n\nConfidence: With the available data, I am moderately confident (0.7) that my proposed solution accurately represents the intended meaning behind the riddle."]
1
[' 

[{'Solution': 'A',
  'Reasoning': "Let us analyze the given information:\n1. There are two farmers: one diligent and the other lazy.\n2. The lazy farmer has a child who turned out to be a hardworking farmer.\n3. It is stated that the lazy farmer isn't the father of the hardworking farmer.\n4. Since they are farmers living together in a household, it suggests a close familial bond.\n\nConsidering the language used and avoiding assumptions outside what was directly revealed, the laziest interpretation aligns with answer choice A – the lazy farmer is indeed the mother of the diligent farmer. As long as the genders aren't specified otherwise, assuming either gender role doesn't violate the text. Hence, without further context, selecting A satisfies all conditions while excluding unnecessary speculation.\n\n",
  'Confidence': 'With the available data, I am moderately confident (0.7) that my proposed solution accurately represents the intended meaning behind the riddle.'},
 {'Solution': 'C',

In [ ]:
finalDF = pd.DataFrame(Structural_Mixtral_R0)

,Solution,Reasoning,Confidence
0,A,Let us analyze the given information:\n1. Ther...,"With the available data, I am moderately confi..."
1,C,"At first glance, it appears that Romeo and Jul...",My confidence level is high (0.9). Although sl...
2,C,"Before your birthday each year, you are always...",I am very confident (1.0) in this answer as th...
3,B,"Generally speaking, in sports organizations, t...",My confidence level is quite high (0.9) due to...
4,C,We know that 5% of the population has an unlis...,I am highly confident (0.95) in this solution ...
...,...,...,...
115,T,Perform calculations systematically to ensure ...,"0.95, as the analysis demonstrates consistency..."
116,C,"When the Internet connection is disrupted, use...","0.95, as this answer aligns with common experi..."
117,B,"Given the statement, the person in considerati...",0.9
118,D,We can analyze this question by calculating th...,0.95


In [ ]:
finalDF

,Solution,Reasoning,Confidence
0,A,Let us analyze the given information:\n1. Ther...,"With the available data, I am moderately confi..."
1,C,"At first glance, it appears that Romeo and Jul...",My confidence level is high (0.9). Although sl...
2,C,"Before your birthday each year, you are always...",I am very confident (1.0) in this answer as th...
3,B,"Generally speaking, in sports organizations, t...",My confidence level is quite high (0.9) due to...
4,C,We know that 5% of the population has an unlis...,I am highly confident (0.95) in this solution ...
...,...,...,...
115,C,To calculate the number of fish caught on the ...,"0.95, as the analysis demonstrates consistency..."
116,C,"When the Internet connection is disrupted, use...","0.95, as this answer aligns with common experi..."
117,B,"Given the statement, the person in considerati...",0.9
118,D,We can analyze this question by calculating th...,0.95


In [ ]:
finalDF.to_csv('./Mixtral_R0.csv')

In [ ]:
mapping = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3}

for index, item in enumerate(finalDF['Solution'].tolist()):
  print(mapping[item])

### Round one

In [ ]:
# import all round 0 answers:
gpt = pd.read_csv("./GPT3.5_R0.csv")
mixtral = pd.read_csv("./Mixtral_R0.csv")
claude = pd.read_csv("./Claude_R0.csv")

In [ ]:
prompt = """Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution: label
Reasoning:
Confidence: confidence value

Question: """

def initial_prompt(item, prompt):

  labels = ["A", "B", "C", "D"]
  prompt += f'{item["question"]}\n'
  for index, choice in enumerate(item["choice_list"]):
    prompt += f'{labels[index]} : {choice}\n'
  return prompt

def generate_prompt(gpt, mixtral, llama, prompt):
  prompts = []

  for index in range(120):
    basePrompt = initial_prompt(SP_test.loc[index], prompt)
    basePrompt += """\nCarefully review the following solutions from other agents as additional
information, and provide your own answer and step-by-step reasoning to the question.

Clearly state which point of view you agree or disagree with and why.\n\n"""

    basePrompt += f"Agent one thinks the Answer is {claude.loc[index]['Solution']}.\nagent one reason: {claude.loc[index]['Reasoning']}\nagent one confidence: {claude.loc[index]['Confidence']}\n"
    basePrompt += f"Agent two thinks the Answer is {mixtral.loc[index]['Solution']}.\nagent two reason: {mixtral.loc[index]['Reasoning']}\nagent two confidence: {mixtral.loc[index]['Confidence']}\n"
    basePrompt += f"Agent three thinks the Answer is {gpt.loc[index]['Solution']}.\nagent three reason: {gpt.loc[index]['Reasoning']}\nagent three confidence: {gpt.loc[index]['Confidence']}\n"
    prompts.append(basePrompt)

  return prompts


In [ ]:
prompts = generate_prompt(gpt, mixtral, claude, prompt)

In [ ]:
for index, pr in enumerate(prompts):
  print(index + 1)
  print(pr)
  print("##################################################################################")

1
Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution: label
Reasoning:
Confidence: confidence value

Question: In a small village, two farmers are working in their fields - a diligent farmer and a lazy farmer. The hardworking farmer is the son of the lazy farmer, but the lazy farmer is not the father of the hardworking farmer. Can you explain this unusual relationship?
A : The lazy farmer is his mother.
B : The lazy farmer is not a responsible father as he is lazy.
C : The diligent farmer devoted himself to the farm and gradually forgot his father.
D : None of above.

Carefully review the following solutions from other agents as additional
information, and provide your own answer and step-by-step reasoning to the question.

Clearly state which point of view you agree or disagree with and why.

Agent one thinks the Answe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import time
mixtralResultsR1 = []
for index, row in SP_test.iterrows():
  # if index + 1 < 99:
  #   continue
  chatbot.switch_llm(0)
  id = chatbot.new_conversation()
  chatbot.change_conversation(id)
  query_result = chatbot.query(prompts[index])
  print(index + 1)
  print(query_result)
  mixtralResultsR1.append({"id" : index+1, "response" : query_result.text})
  print()
  time.sleep(2)

1
 Solution: A
Reasoning: The relationship between the hardworking farmer and the lazy farmer seems unusual because typically, we might associate the father figure with farming. However, there is no explicit mention of the genders of these individuals in the question. Given that the lazy farmer is the parent of the hardworking farmer but not the father, the most straightforward explanation is that the lazy farmer is the mother. This conclusion does not require making any assumptions beyond the information provided.
Confidence: 0.95

I concur with Agent One and Agent Three regarding their analysis of the situation. I appreciate Agent Two's emphasis on avoiding unnecessary speculations based on traditional roles; however, I believe my suggested confidence level reflects a higher degree of certainty since the relationships mentioned in the problem do not inherently imply specific gender roles for the individuals involved.

2
 Solution: C
Reasoning: The presence of glass shards and water o

In [ ]:
# save dataframe:
mixtralR1 = pd.DataFrame(mixtralResultsR1)
mixtralR1.to_csv("./MixtralR1.csv")

In [ ]:
mixtralR1 = pd.read_csv("./MixtralR1.csv")

In [ ]:
# create structural data frame:
Structural_Mixtral_R1 = []

for index, item in mixtralR1.iterrows():
  print(index)
  print(item['response'].split('Solution: '))

  solution = item['response'].split('Solution: ')[1][0]
  reasoning = item['response'].split('Reasoning:')[1].split('Confidence:')[0]
  confidence = item['response'].split('Confidence:')[1].split('\n')[0].strip()

  Structural_Mixtral_R1.append({"Solution" : solution, "Reasoning" : reasoning, "Confidence" : confidence})

Structural_Mixtral_R1

0
[' ', "A\nReasoning: The relationship between the hardworking farmer and the lazy farmer seems unusual because typically, we might associate the father figure with farming. However, there is no explicit mention of the genders of these individuals in the question. Given that the lazy farmer is the parent of the hardworking farmer but not the father, the most straightforward explanation is that the lazy farmer is the mother. This conclusion does not require making any assumptions beyond the information provided.\nConfidence: 0.95\n\nI concur with Agent One and Agent Three regarding their analysis of the situation. I appreciate Agent Two's emphasis on avoiding unnecessary speculations based on traditional roles; however, I believe my suggested confidence level reflects a higher degree of certainty since the relationships mentioned in the problem do not inherently imply specific gender roles for the individuals involved."]
1
[' ', "C\nReasoning: The presence of glass shards and water on 

[{'Solution': 'A',
  'Reasoning': ' The relationship between the hardworking farmer and the lazy farmer seems unusual because typically, we might associate the father figure with farming. However, there is no explicit mention of the genders of these individuals in the question. Given that the lazy farmer is the parent of the hardworking farmer but not the father, the most straightforward explanation is that the lazy farmer is the mother. This conclusion does not require making any assumptions beyond the information provided.\n',
  'Confidence': '0.95'},
 {'Solution': 'C',
  'Reasoning': " The presence of glass shards and water on the floor suggests that there was a container made of glass that held liquid and broke. The options A and B suggest scenarios where either Romeo or Juliet died out of fright or due to the bookcase falling onto them, but these explanations don't account for the glass shards and water. On the other hand, option C explains that Romeo and Juliet were actually fish

In [ ]:
finalDF = pd.DataFrame(Structural_Mixtral_R1)

In [ ]:
finalDF

,Solution,Reasoning,Confidence
0,A,The relationship between the hardworking farm...,0.95
1,C,The presence of glass shards and water on the...,0.95
2,C,"Every year, before your birthday, you are ind...",0.95
3,B,According to all the information provided by ...,0.95
4,C,The problem states that 5% of the city's popu...,1.0
...,...,...,...
115,C,Agent two correctly pointed out that we need ...,I am quite confident in my solution since it r...
116,C,"Based on the information provided, it is high...",0.98
117,B,"The phrase ""He has wed numerous women, but ne...",1.0
118,B,The question asks for the number of legs on t...,0.98


In [ ]:
finalDF.to_csv('./Mixtral_R1.csv')

In [ ]:
mapping = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3}

for index, item in enumerate(finalDF['Solution'].tolist()):
  print(mapping[item])

0
2
2
1
2
0
2
2
3
3
1
0
2
0
1
1
0
2
2
0
0
2
1
0
0
2
0
2
1
1
1
2
2
2
2
2
1
0
2
2
1
1
0
0
1
0
2
2
0
0
0
0
1
0
1
0
0
1
1
1
0
1
1
1
3
0
0
1
2
0
2
0
3
2
1
2
0
1
1
0
3
0
1
0
3
1
2
3
1
2
3
2
2
0
2
1
1
2
0
1
1
1
0
2
2
2
3
1
2
1
1
0
0
2
0
2
2
1
1
2


### Round 2

In [ ]:
# import all round 1 answers:
gpt = pd.read_csv("./GPT3.5_R1.csv")
mixtral = pd.read_csv("./Mixtral_R1.csv")
claude = pd.read_csv("./Claude_R1.csv")

In [ ]:
prompt = """Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution: label
Reasoning:
Confidence: confidence value

Question: """

def initial_prompt(item, prompt):

  labels = ["A", "B", "C", "D"]
  prompt += f'{item["question"]}\n'
  for index, choice in enumerate(item["choice_list"]):
    prompt += f'{labels[index]} : {choice}\n'
  return prompt

def generate_prompt(gpt, mixtral, llama, prompt):
  prompts = []

  for index in range(120):
    basePrompt = initial_prompt(SP_test.loc[index], prompt)
    basePrompt += """\nCarefully review the following solutions from other agents as additional
information, and provide your own answer and step-by-step reasoning to the question.

Clearly state which point of view you agree or disagree with and why.\n\n"""

    basePrompt += f"Agent one thinks the Answer is {claude.loc[index]['Solution']}.\nagent one reason: {claude.loc[index]['Reasoning']}\nagent one confidence: {claude.loc[index]['Confidence']}\n"
    basePrompt += f"Agent two thinks the Answer is {mixtral.loc[index]['Solution']}.\nagent two reason: {mixtral.loc[index]['Reasoning']}\nagent two confidence: {mixtral.loc[index]['Confidence']}\n"
    basePrompt += f"Agent three thinks the Answer is {gpt.loc[index]['Solution']}.\nagent three reason: {gpt.loc[index]['Reasoning']}\nagent three confidence: {gpt.loc[index]['Confidence']}\n"
    prompts.append(basePrompt)

  return prompts


In [ ]:
prompts = generate_prompt(gpt, mixtral, claude, prompt)

In [ ]:
for index, pr in enumerate(prompts):
  print(index + 1)
  print(pr)
  print("##################################################################################")

1
Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution: label
Reasoning:
Confidence: confidence value

Question: In a small village, two farmers are working in their fields - a diligent farmer and a lazy farmer. The hardworking farmer is the son of the lazy farmer, but the lazy farmer is not the father of the hardworking farmer. Can you explain this unusual relationship?
A : The lazy farmer is his mother.
B : The lazy farmer is not a responsible father as he is lazy.
C : The diligent farmer devoted himself to the farm and gradually forgot his father.
D : None of above.

Carefully review the following solutions from other agents as additional
information, and provide your own answer and step-by-step reasoning to the question.

Clearly state which point of view you agree or disagree with and why.

Agent one thinks the Answe

In [ ]:
import time
mixtralResultsR2 = []
for index, row in SP_test.iterrows():
  # if index + 1 < 99:
  #   continue
  chatbot.switch_llm(0)
  id = chatbot.new_conversation()
  chatbot.change_conversation(id)
  query_result = chatbot.query(prompts[index])
  print(index + 1)
  print(query_result)
  mixtralResultsR2.append({"id" : index+1, "response" : query_result.text})
  print()
  time.sleep(2)

1
 Solution: A
Reasoning: The relationships mentioned in the question suggest that the lazy farmer cannot be the father of the diligent farmer since it has been explicitly stated that they are not. Thus, if the lazy farmer is indeed a parent to the diligent farmer, then the only remaining option is for them to be the mother. There is nothing indicating nontraditional gender roles; therefore, we should stick to typical family structures unless specified otherwise.
Confidence: 0.95

2
 Solution: C
Reasoning: The mention of glass shards and water on the floor indicates that there must have been a broken container containing liquid. Options A and B do not explain the presence of these elements. However, option C provides a reasonable explanation that Romeo and Juliet were pet fish residing in an aquarium placed on the bookshelf. The vibration from the passing train could have tipped over the aquarium causing it to break and result in the death of the fish. This interpretation aligns with a

In [ ]:
# save dataframe:
mixtralR2 = pd.DataFrame(mixtralResultsR2)
mixtralR2.to_csv("./MixtralR2.csv")

In [ ]:
# create structural data frame:
Structural_Mixtral_R2 = []

for index, item in mixtralR2.iterrows():
  print(index)
  print(item['response'].split('Solution: '))

  solution = item['response'].split('Solution: ')[1][0]
  reasoning = item['response'].split('Reasoning:')[1].split('Confidence:')[0]
  if index == 7:
    confidence = '0.95'
  else:
    confidence = item['response'].split('Confidence:')[1].split('\n')[0].strip()

  Structural_Mixtral_R2.append({"Solution" : solution, "Reasoning" : reasoning, "Confidence" : confidence})

Structural_Mixtral_R2

0
[' ', 'A\nReasoning: The relationships mentioned in the question suggest that the lazy farmer cannot be the father of the diligent farmer since it has been explicitly stated that they are not. Thus, if the lazy farmer is indeed a parent to the diligent farmer, then the only remaining option is for them to be the mother. There is nothing indicating nontraditional gender roles; therefore, we should stick to typical family structures unless specified otherwise.\nConfidence: 0.95']
1
[' ', "C\nReasoning: The mention of glass shards and water on the floor indicates that there must have been a broken container containing liquid. Options A and B do not explain the presence of these elements. However, option C provides a reasonable explanation that Romeo and Juliet were pet fish residing in an aquarium placed on the bookshelf. The vibration from the passing train could have tipped over the aquarium causing it to break and result in the death of the fish. This interpretation aligns with all t

[{'Solution': 'A',
  'Reasoning': ' The relationships mentioned in the question suggest that the lazy farmer cannot be the father of the diligent farmer since it has been explicitly stated that they are not. Thus, if the lazy farmer is indeed a parent to the diligent farmer, then the only remaining option is for them to be the mother. There is nothing indicating nontraditional gender roles; therefore, we should stick to typical family structures unless specified otherwise.\n',
  'Confidence': '0.95'},
 {'Solution': 'C',
  'Reasoning': ' The mention of glass shards and water on the floor indicates that there must have been a broken container containing liquid. Options A and B do not explain the presence of these elements. However, option C provides a reasonable explanation that Romeo and Juliet were pet fish residing in an aquarium placed on the bookshelf. The vibration from the passing train could have tipped over the aquarium causing it to break and result in the death of the fish. Th

In [ ]:
finalDF = pd.DataFrame(Structural_Mixtral_R2)

In [ ]:
finalDF

,Solution,Reasoning,Confidence
0,A,The relationships mentioned in the question s...,0.95
1,C,The mention of glass shards and water on the ...,0.95
2,C,"The statement ""before your birthday you'd be ...",0.98
3,B,All three agents have provided consistent rea...,0.98
4,C,The question asks for how many individuals ou...,1.0
...,...,...,...
115,C,"To solve this problem, we can use reverse eng...",0.95
116,C,"According to the passage, ""Bob was working on...",0.95
117,B,"The statement ""he has wed numerous women, but...",1.0
118,B,"According to the information provided, the be...",1.0


In [ ]:
finalDF.to_csv('./Mixtral_R2.csv')

In [ ]:
mapping = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3}

for index, item in enumerate(finalDF['Solution'].tolist()):
  print(mapping[item])

0
2
2
1
2
0
0
1
0
3
1
0
2
0
1
1
0
2
2
0
0
2
1
0
0
2
0
2
0
1
1
2
2
2
2
2
1
0
2
2
1
1
0
0
1
0
2
2
0
0
0
0
2
0
1
0
0
1
1
3
0
1
1
1
3
0
2
1
2
0
2
0
3
2
1
2
0
1
1
0
3
0
1
0
3
1
2
3
3
2
3
2
3
0
2
1
1
2
0
1
1
3
0
2
2
2
0
1
2
1
1
0
0
2
0
2
2
1
1
2
